In [9]:
import pprint
from pymongo import MongoClient
db = MongoClient().aggregation_example

In [6]:
from bson.son import SON
pipeline = [
     {"$unwind": "$tags"},
     {"$group": {"_id": "$tags", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
 ]
list(db.things.aggregate(pipeline))

[{'_id': 'cat', 'count': 3},
 {'_id': 'dog', 'count': 2},
 {'_id': 'mouse', 'count': 1}]

In [7]:
from bson.code import Code
mapper = Code("""
              function () {
                this.tags.forEach(function(z) {
                  emit(z, 1);
                });
              }
              """)

reducer = Code("""
               function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                   total += values[i];
                 }
                 return total;
               }
               """)


In [10]:
result = db.things.map_reduce(mapper, reducer, "myresults")
for doc in result.find():
  pprint.pprint(doc)


{'_id': 'cat', 'value': 3.0}
{'_id': 'dog', 'value': 2.0}
{'_id': 'mouse', 'value': 1.0}


In [12]:
pprint.pprint(
    db.things.map_reduce(mapper, reducer, "myresults", full_response=True))


{'counts': {'emit': 6, 'input': 4, 'output': 3, 'reduce': 2},
 'ok': 1.0,
 'result': 'myresults',
 'timeMillis': 83}


In [13]:
results = db.things.map_reduce(
    mapper, reducer, "myresults", query={"x": {"$lt": 2}})
for doc in results.find():
  pprint.pprint(doc)


{'_id': 'cat', 'value': 1.0}
{'_id': 'dog', 'value': 1.0}


In [14]:

from bson.son import SON
pprint.pprint(
    db.things.map_reduce(
        mapper,
        reducer,
        out=SON([("replace", "results"), ("db", "outdb")]),
        full_response=True))

{'counts': {'emit': 6, 'input': 4, 'output': 3, 'reduce': 2},
 'ok': 1.0,
 'result': {'collection': 'results', 'db': 'outdb'},
 'timeMillis': 71}
